# Neuronale Netze mit Python und TensorFlow

Im Gegensatz zu vorigen Artikel über TensorFlow, handelt sich hier eher um eine Anleitung, wie man TensorFlow verwendet um ein neuronales Netz zu erstellen. Der Artikel geht davon aus, dass man über Python-Kentnisse verfügt und TensorFlow bereits installiert hat. [(Wie installiert man TensorFlow)](https://www.tensorflow.org/install/)

Wir fangen an mit einem Vergleich zwischen echten und künstlichen neuronale Netze:

## Echte Neuronale Netzwerke

![nervenzelle](images/neuron.png)

*Nervenzelle; Autor: [Quasar Jarosz](https://en.wikipedia.org/wiki/User:Quasar_Jarosz); Licenz: [CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0)*

Das ist ein Neuron. Im menschlichen Gehirn gibt es 86 Milliarden davon. Die wichtigsten Strukturen sind:
- **Dendrite** - erhalten Signale von anderen Nervenzellen. Ein Neuron kann sehr viele Dendrite haben
- **Zellkörper** - summiert die Signale um Ausgabe zu generieren
- **Axon** - wenn die Summe einen Schwellwert erreicht, wird ein Signal über den Axon übertragen. Nervenzelle haben immer nur einen Axon
- **Axonterminale (Synapse)?** - Die Verbindungspunkt zwischen Axon und Dendriten. Die Stärke der Verbindung entspricht der Stärke der Stärke des übertragten Signal (synaptische Gewichte)

## Künstliche Neuronale Netz

Perzeptronen und Sigmoidneurone sind die Hauptbestandteile eines neuronalen Netzes. 

![Perzeptrone](images/perceptron.png)

*Bild von Perzeptronen. Ein mit, der andere ohne Bias*

Ähnlich wie ein Neuron, haben Perzeptronen mehrere **Inputs** (*x*) mit entsprechende **Gewichtungen** (*w*). Die Inputs sind immer 0 oder 1. Jedes Input wird mit einer der Gewichtungen multipliziert und man addiert noch ein **Bias** (*b oder Theta*) dazu. Am Ende summiert man alle Ergebnisse und verwendet einen **Schwellwert** um zu entscheiden, ob 1 oder 0 ausgegeben wird.

Perzeptronen sind oft zu primitiv für künstliche neuronale Netze, deswegen benutzt man Sigmoidneuronen. Die Unterschied ist, dass sie auch eine **Aktivierungsfunktion** haben. 

![Sigmoidfunktion](images/sigmoid_fn.png)


Diese Funktion erlaubt, dass diese Sigmoidneuronen als Inputs Zahlen zwischen 0 und 1 bekommen. Das hilft so, dass wenn man kleine Anpassungen an den Gewichtungen und Biasen macht, dann gibt es auch kleine Unterschiede bei der Ausgabe.

![neuronales Netz](images/neural_network.png)

*Neuronales Netz von [Glosser.ca](https://commons.wikimedia.org/wiki/User_talk:Glosser.ca), lizensiert unter [CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0)*

Ein neuronales Netz besteht aus mehrere Schichten und jede Schicht hat mehrere Neuronen, wobei in der Regel hat jedes Neuron aus einer Schicht eine Verbindung zu allen anderen Neuronen aus der nächste Schicht. Die erste Schicht ist die Inputschicht, wo man sein Datensatz eingibt. Es kann beliebig viele Zwischenschichten geben, aber die letzte ist die Outputschicht, mit 1 oder mehreren Neuronen (Die Anzahl hängt vom Problem, den man lösen möchte ab). 

## Funktionsweise von TensorFlow

- Tensor, Constant Variable
- DataFlow graph

## Implementierung eines neuronalen Netzes

Wir werden ein eifaches neuronales Netz mit TensorFlow implementieren. Zusätzlich brauchen wir aber einige Funktionen aus *scikit-learn*. Wir werden den [Iris-Datensatz](https://en.wikipedia.org/wiki/Iris_flower_data_set) benutzen und versuchen mit dem neuronalen Netz die Blumen richtig zu klassifizieren.

Zum ersten wollen wir Tensorflow und einige Funktionen aus *scikit-learn* importieren. Dann laden wir den Datensatz.

In [2]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Datensatz
data = load_iris()
features = data.data
labels = data.target.reshape((-1, 1))

# Klassen von Blumen zeigen
labels

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

### One-Hot Encoding

Wie man hier sieht, sind die Blummenklassen mit 0, 1 oder 2 bezeichnet. In der Regel, wenn man neuronale Netze für Klassifizierungsprobleme benutzen will, hat man so viele Neuronen in der letzten Schicht wie Klassen. Wir erwarten für jede Stichprobe eine Ausgabe, die so Aussieht `[0.15, 0.70, 0.15]`. In diesem Beispiel ist die Klasse der Blume `1`, weil die zweite Zahl die größte ist.  

Wir wollen aber die Vorhersagen des neuronalen Netzes mit den echten Klassen vergleichen, deswegen wandeln wir die numerische Darstellung der Klassen zur sogenannten *One-hot encoding*. So hat man eine 3-Array als Bezeichner für jede Klasse.

Beispiel:
```
Klasse 0 -> [1, 0, 0]
Klasse 1 -> [0, 1, 0]
Klasse 2 -> [0, 0, 1]
```

Wir verwenden dafür die `OneHotEncoder` von *scikit-learn*

In [3]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
enc.fit_transform(labels)

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.

### Trainings- und Testdaten

Um das Modell gut validieren zu können, sollen wir Daten verwenden, mit denen nie trainiert worden ist. Deswegen benutzen wir die `train_test_split` Funktion, um 20 Prozent der Daten als Testdaten zu nehmen. Dann merken wir uns die Anzahl von Merkmale bzw. von Klassen. In diesem Fall ist `x_size = 4` und `y_size = 3`

In [4]:
# Trainings- und Testdaten erzeugen
train_x, test_x, train_y, test_y = train_test_split(
                                features, 
                                enc.transform(labels) )

x_size = train_x.shape[1]
y_size = train_y.shape[1]

**Placeholder** (Platzhalter) sind Tensors, die für die Dateneingabe sorgen. Man muss nur die Dimensionen spezifizieren. Zum Beispiel sind hier die Dimensionen für *X* `[None, x_size]`, weil wir eine unbestimmte Zahl von Stichproben haben mit jeweils `x_size` Merkmalen.

In *X* geben wir die Merkmale ein, und in *Y* geben wir die korrekte Klassen ein, damit wir das Modell trainieren können.

In [5]:
X = tf.placeholder(tf.float32, shape=[None, x_size])
Y = tf.placeholder(tf.float32, shape=[None, y_size])


Für das Modell des neuronales Netzes benutzen wir 2 Zwischenschichten, mit jeweils 128 Neuronen. Wir benutzen verschiedene Aktivierungsfunktionen für die Zwischen- und Ausgabeschichten. Für die Ausgabeschicht benutzen wir die `softmax` Aktivierungsfunktion. Diese erlaubt uns eine Wahrscheinlichkeit für die Ausgabeklassen zu definieren, da die Summe aller Elemente aus der Ausgabeliste wegen der Aktivierungsfunktion gleich `1` ist. Beispiel für Ausgabe: `[0.10, 0.78. 0.12]`. Das heißt, dass das Modell 78% sicher ist, dass die aktuelle Stichprobe der Klasse `1` ist.

Die `softmax` Funktion funktioniert nicht so gut für die Zwischenschichten, deswegen haben wir die `sigmoid` Aktivierungsfunktion ausgewählt.

TensorFlow bietet viele Werkzeuge an, mit denen man ein Modell erstellen kann. Wir schauen uns hier zwei Möglichkeiten. Zum ersten definieren wir die Schichten mit TensorFlow Core. Man braucht mehr Codezeilen, dafür aber hat man mehr Kontrolle über das Program. Die zweite Lösung verwendet die API `tf.layers`, womit man schnell neue Schichten definieren kann.

#### TensorFlow Core
Um schneller das Modell erstellen zu können, definieren wir eine eigene Funktion `hidden_layer`, mit den folgenden Parametern.
- `t_input` - Die Eingabetensor oder auch die vorige Schicht
- `w_shape` - Eine 2-Array mit den Dimensionen der Schicht. Das erste Element ist die Neuronenanzahl der vorigen Schicht und das zweite - die Neuronenanzahl dieser Schicht.
- `activation` - Aktivierungsfunktion. Da wir für verschiedene Schichten, unterschiedliche Aktivierungsfunktionen verwenden wollen, brauchen wir diese als Parameter einzugeben.

Mit `random_normal(shape)` generieren wir zufällige Werte mit bestimmten Dimensionen für Gewichtungen und Biases. Danach bilden wir Unbekannten, deren Werte man mit einem Optimierer anpassen kann. 

Die algebraische Operationen, die im neuronalen Netz stattfinden, sind folgenderweise definiert:
- man multipliziert das Input mit den Gewichtungen
- man addiert die Biases dazu
- man wendet die Aktivierungsfunktion an das Ergebnis an

Man bemerkt, dass die Dimensionen der Gewichtungen und der Biases sich unterschieden. In unserem Fall hat das Input Dimensionen `1x4`, weil jede Stichprobe 4 Merkmale hat. Die Gewichtungen haben die Dimension `4x128`, weil wir 128 Neuronen haben und in jedem sollen wir eine Gewichtung pro Merkmal haben. Nachdem wir `tf.matmul()` ausführen, kommt ein Ergebnis heraus mit den Dimensionen `1x128`. Um die Biases dazu zu addieren, brauchen die dieselbe Dimensionen zu haben. 


#### `tf.layers`

Mit `tf.layers` ist es sehr einfach ein neuronales Netz zu definieren. `tf.layers.dense` bietet uns das typische Schichtenmodell und behandelt automatisch die Addition von Biases und die Berechnung von Dimensionen. Wir geben nur das Eingabetensor, die Neuronenanzahl und die Aktivierungsfunktion ein. 

Es gibt auch andere Parameter, die man anpassen kann, die aber für uns im Moment nicht relevant sind. Bei Interesse, kann man sich die Dokumentation anschauen.

In [6]:
# NN Modell mit TF Core
def hidden_layer(t_input, w_shape, activation=tf.nn.sigmoid):
    weights = tf.Variable(tf.random_normal(w_shape))
    biases = tf.Variable(tf.random_normal([1, w_shape[1]]))
    
    return activation(tf.add(tf.matmul(t_input, weights), biases))

h_layer1 = hidden_layer(X, [x_size, 128])
h_layer2 = hidden_layer(h_layer1, [128, 128])
y_hat = hidden_layer(h_layer2, [128, y_size], tf.nn.softmax)

In [7]:
# MM Modell mit tf.layers
h_layer1 = tf.layers.dense(X, 128, activation=tf.nn.sigmoid)
h_layer2 = tf.layers.dense(h_layer1, 128, activation=tf.nn.sigmoid)
y_hat = tf.layers.dense(h_layer2, y_size, activation=tf.nn.softmax)

### Kostenfunktion, Optimierer und Session

Wir brauchen jetzt eine **Kostenfunktion** um den Fehler zwischen den Vorhersgen und die echten Klassen zu berechnen. Die Tensorflow Funktion `softmax_cross_entropy_with_logits()` ist eine sehr gute Kostenfunktion, wenn man ein Klassifikationsproblem hat. Die Rückgabewert dieser Funktion ist eine Liste, mit den Fehlern der Vorhersage. Wir verwenden dann die Funktion `reduce_mean()` um den Mittelwert dieser Fehler zu berechnen, um nur mit einer Zahl zu arbeiten.

Danach brauchen wir einen Optimierer, damit wir die Gewichtungen in den vorigen Schichten anpassen zu können. Den `GradientDescentOptimizer` kann man hier gut anwenden und das Ziel ist die Kostenfunktion zu minimieren. Als argument gibt man die Lernrate ein, wobei man darauf Achten muss, dass eine zu große Zahl zu einem ungenaueren Modell führen kann, und eine zu kleine Zahl zu lange Trainingszeiten führen kann.

Bevor wir mit der Ausführung des Programms startet, müssen wir die Unbekannten initializieren. Diese sind die Gewichtungen und Biases in den Neuronenschichten, die der Optimierer anpasst, um die Kostenfunktion zu minimieren. Die Unbekannten haben am Anfang keine Werte und wir müssen den `global_variables_initializer()` ausführen um zufällige Werte da zu setzen.

In [8]:
# Kosten- und Optimierungsfunktion
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=Y, logits=y_hat))

train_step = tf.train.GradientDescentOptimizer(0.005).minimize(loss)

# Unbekannten initialisieren und Session erstellen
sess = tf.Session()
sess.run(tf.global_variables_initializer())

### Trainingsschleife

Die erste Schleife ist für die Anzahl der Epochen. In diesem Fall wollen wir 300 mal alle Stichproben dem neuronalen Netz eingeben und zwar einer nach dem anderen (die zweite Schleife).

Zum ersten führen wir `session.run()` mit `train_step` aus, also man berechnet die Kostenfunktion und den Optimierer. Deswegen geben wir die Trainingsdaten ein und alle 10 Epochen wollen wir die Genauigkeit evaluiren.

Dafür verwenden wir `tf.argmax` um die Stelle der größten Zahl bei der Vorhersage und bei den echten Klassen zu vergleichen. Das gibt uns eine Liste von booleschen Werten (`True` oder `False`). Mithilfe von `cast(tf.float32)`, wandeln wir die boolesche Werte in `1` oder `0` um. Schließlich berechnen wir den Mittelwert dieser Liste. So bekommen wir eine Genauigkeit als Prozent. 

In [9]:
# Training Loop
for epoch in range(300):
    for i in range(train_x.shape[0]):
        sess.run(train_step, feed_dict={X: train_x[i:i+1], Y: train_y[i: i+1]})                                                 
        
    if (epoch % 10 == 0):
        correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(Y, 1))                                                      
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))                                                       
        print("Epoch %d: Accuraccy = %f, Loss = %f" % (                                                                          
                          epoch,                                                                                                   
                          sess.run(accuracy, feed_dict={X: test_x, Y: test_y}),                                                    
                          sess.run(loss, feed_dict={X: train_x, Y: train_y})))
    

Epoch 0: Accuraccy = 0.263158, Loss = 1.098506
Epoch 10: Accuraccy = 0.552632, Loss = 1.074560
Epoch 20: Accuraccy = 0.552632, Loss = 1.044309
Epoch 30: Accuraccy = 0.552632, Loss = 0.995192
Epoch 40: Accuraccy = 0.552632, Loss = 0.933063
Epoch 50: Accuraccy = 0.552632, Loss = 0.885879
Epoch 60: Accuraccy = 0.552632, Loss = 0.857331
Epoch 70: Accuraccy = 0.552632, Loss = 0.829077
Epoch 80: Accuraccy = 0.578947, Loss = 0.803954
Epoch 90: Accuraccy = 0.736842, Loss = 0.781753
Epoch 100: Accuraccy = 0.789474, Loss = 0.761336
Epoch 110: Accuraccy = 0.894737, Loss = 0.741924
Epoch 120: Accuraccy = 0.894737, Loss = 0.723349
Epoch 130: Accuraccy = 0.921053, Loss = 0.705859
Epoch 140: Accuraccy = 0.921053, Loss = 0.689829
Epoch 150: Accuraccy = 0.921053, Loss = 0.675535
Epoch 160: Accuraccy = 0.947368, Loss = 0.663070
Epoch 170: Accuraccy = 0.947368, Loss = 0.652366
Epoch 180: Accuraccy = 0.947368, Loss = 0.643252
Epoch 190: Accuraccy = 0.973684, Loss = 0.635516
Epoch 200: Accuraccy = 0.973684

Man sieht, dass die Genauigkeit steigt und die Kostenfunktion sinkt. Man kann auch die Lernrate und die Anzahl von Neuronen in einer Schicht anpassen um die Veränderungen in der Genaigkeit sich anzuschauen.

### Was kann man verbessern?

- Man kann immer den Anzahl von Schichten und Neuronen anpassen
- die Lernrate kann auch angepasst werden
- um bessere Modelle zu bekommen, soll man die Daten bei jeder Epoche schlurfen
- für bessere Laufzeit, soll man die Stichproben nicht eine nach den anderen dem neuronalen Netzes eingeben, sondern die in Stapel zusammenfassen. Die größe der Stapel hängt von der Anzahl von Merkmalen und dem vorhandenen Haupt- oder Videospeicherplatz ab.

### Ressoursen

- Git Repository mit dem kompletten Code - [Link]()
- Wie wählt man eine optimale Anzahl von Zwischenschichten und deren Nuronen? (Enghlisch) - [Link](https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw)